# Ungraded Lab: Build a Multi-output Model

In this lab, we'll show how you can build models with more than one output. The dataset we will be working on is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency). It is an Energy Efficiency dataset which uses the bulding features (e.g. wall area, roof area) as inputs and has two outputs: Cooling Load and Heating Load. Let's see how we can build a model to train on this data.

## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

## Utilities

We define a few utilities for data conversion and visualization to make our code more neat.

In [2]:
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2


def norm(x):
    return (x - train_stats['mean']) / train_stats['std']


def plot_diff(y_true, y_pred, title=''):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    plt.plot([-100, 100], [-100, 100])
    plt.show()


def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color='blue', label=metric_name)
    plt.plot(history.history['val_' + metric_name], color='green', label='val_' + metric_name)
    plt.show()

## Prepare the Data

We download the dataset and format it for training.

In [3]:
# Specify data URI
URI = './data/ENB2012_data.xlsx'

# Use pandas excel reader
df = pd.read_excel(URI)
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into train and test with 80 train / 20 test
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

# Get Y1 and Y2 as the 2 outputs and format them as np arrays
train_stats.pop('Y1')
train_stats.pop('Y2')
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

# Normalize the training and test data
norm_train_X = norm(train)
norm_test_X = norm(test)

## Build the Model

Here is how we'll build the model using the functional syntax. Notice that we can specify a list of outputs (i.e. `[y1_output, y2_output]`) when we instantiate the `Model()` class.

In [4]:
# Define model layers.
input_layer = Input(shape=(len(train .columns),))
first_dense = Dense(units='128', activation='relu')(input_layer)
second_dense = Dense(units='128', activation='relu')(first_dense)

# Y1 output will be fed directly from the second dense
y1_output = Dense(units='1', name='y1_output')(second_dense)
third_dense = Dense(units='64', activation='relu')(second_dense)

# Y2 output will come via the third dense
y2_output = Dense(units='1', name='y2_output')(third_dense)

# Define the model with the input layer and a list of output layers
model = Model(inputs=input_layer, outputs=[y1_output, y2_output])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          1152        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
______________________________________________________________________________________________

## Configure parameters

We specify the optimizer as well as the loss and metrics for each output.

In [5]:
# Specify the optimizer, and compile the model with loss functions for both outputs
optimizer = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

## Train the Model

In [ ]:
# Train the model for 500 epochs
history = model.fit(norm_train_X, train_Y,
                    epochs=500, batch_size=10, validation_data=(norm_test_X, test_Y))

Train on 614 samples, validate on 154 samples
Epoch 1/500
614/614 [==============================] - 1s 1ms/sample - loss: 277.4080 - y1_output_loss: 129.6350 - y2_output_loss: 146.0509 - y1_output_root_mean_squared_error: 11.4190 - y2_output_root_mean_squared_error: 12.1249 - val_loss: 38.2426 - val_y1_output_loss: 13.1363 - val_y2_output_loss: 23.9068 - val_y1_output_root_mean_squared_error: 3.6902 - val_y2_output_root_mean_squared_error: 4.9624
Epoch 2/500
614/614 [==============================] - 0s 149us/sample - loss: 34.1060 - y1_output_loss: 13.1224 - y2_output_loss: 20.9294 - y1_output_root_mean_squared_error: 3.6259 - y2_output_root_mean_squared_error: 4.5781 - val_loss: 29.8416 - val_y1_output_loss: 9.1939 - val_y2_output_loss: 20.8331 - val_y1_output_root_mean_squared_error: 3.0442 - val_y2_output_root_mean_squared_error: 4.5359
Epoch 3/500
614/614 [==============================] - 0s 152us/sample - loss: 26.9582 - y1_output_loss: 10.2332 - y2_output_loss: 16.7781 - y1_ou

Epoch 42/500
614/614 [==============================] - 0s 148us/sample - loss: 7.4215 - y1_output_loss: 2.4099 - y2_output_loss: 4.9651 - y1_output_root_mean_squared_error: 1.5546 - y2_output_root_mean_squared_error: 2.2371 - val_loss: 6.2319 - val_y1_output_loss: 2.1194 - val_y2_output_loss: 4.2407 - val_y1_output_root_mean_squared_error: 1.4590 - val_y2_output_root_mean_squared_error: 2.0256
Epoch 43/500
614/614 [==============================] - 0s 142us/sample - loss: 8.0111 - y1_output_loss: 2.5540 - y2_output_loss: 5.3915 - y1_output_root_mean_squared_error: 1.6050 - y2_output_root_mean_squared_error: 2.3314 - val_loss: 6.3777 - val_y1_output_loss: 2.2702 - val_y2_output_loss: 4.0412 - val_y1_output_root_mean_squared_error: 1.5319 - val_y2_output_root_mean_squared_error: 2.0078
Epoch 44/500
614/614 [==============================] - 0s 143us/sample - loss: 6.9168 - y1_output_loss: 2.1787 - y2_output_loss: 4.7201 - y1_output_root_mean_squared_error: 1.4802 - y2_output_root_mean_s

Epoch 63/500
614/614 [==============================] - 0s 145us/sample - loss: 4.3877 - y1_output_loss: 1.1679 - y2_output_loss: 3.1794 - y1_output_root_mean_squared_error: 1.0859 - y2_output_root_mean_squared_error: 1.7912 - val_loss: 3.4099 - val_y1_output_loss: 0.9814 - val_y2_output_loss: 2.4239 - val_y1_output_root_mean_squared_error: 1.0061 - val_y2_output_root_mean_squared_error: 1.5485
Epoch 64/500
614/614 [==============================] - 0s 148us/sample - loss: 5.2845 - y1_output_loss: 1.4054 - y2_output_loss: 3.8500 - y1_output_root_mean_squared_error: 1.1902 - y2_output_root_mean_squared_error: 1.9667 - val_loss: 3.8424 - val_y1_output_loss: 1.2152 - val_y2_output_loss: 2.6514 - val_y1_output_root_mean_squared_error: 1.1019 - val_y2_output_root_mean_squared_error: 1.6212
Epoch 65/500
614/614 [==============================] - 0s 153us/sample - loss: 6.1393 - y1_output_loss: 1.9394 - y2_output_loss: 4.1560 - y1_output_root_mean_squared_error: 1.3944 - y2_output_root_mean_s

Epoch 84/500
614/614 [==============================] - 0s 143us/sample - loss: 2.8276 - y1_output_loss: 0.6531 - y2_output_loss: 2.1568 - y1_output_root_mean_squared_error: 0.8095 - y2_output_root_mean_squared_error: 1.4739 - val_loss: 2.1302 - val_y1_output_loss: 0.4563 - val_y2_output_loss: 1.6971 - val_y1_output_root_mean_squared_error: 0.6795 - val_y2_output_root_mean_squared_error: 1.2917
Epoch 85/500
614/614 [==============================] - 0s 139us/sample - loss: 3.7129 - y1_output_loss: 1.0400 - y2_output_loss: 2.6975 - y1_output_root_mean_squared_error: 1.0236 - y2_output_root_mean_squared_error: 1.6325 - val_loss: 3.6693 - val_y1_output_loss: 0.4557 - val_y2_output_loss: 3.2527 - val_y1_output_root_mean_squared_error: 0.6821 - val_y2_output_root_mean_squared_error: 1.7900
Epoch 86/500
614/614 [==============================] - 0s 141us/sample - loss: 4.6908 - y1_output_loss: 1.3023 - y2_output_loss: 3.3538 - y1_output_root_mean_squared_error: 1.1456 - y2_output_root_mean_s

Epoch 105/500
614/614 [==============================] - 0s 139us/sample - loss: 6.7118 - y1_output_loss: 2.1111 - y2_output_loss: 4.5746 - y1_output_root_mean_squared_error: 1.4572 - y2_output_root_mean_squared_error: 2.1421 - val_loss: 2.8006 - val_y1_output_loss: 0.4955 - val_y2_output_loss: 2.3430 - val_y1_output_root_mean_squared_error: 0.7055 - val_y2_output_root_mean_squared_error: 1.5175
Epoch 106/500
614/614 [==============================] - 0s 139us/sample - loss: 2.2238 - y1_output_loss: 0.5734 - y2_output_loss: 1.6409 - y1_output_root_mean_squared_error: 0.7593 - y2_output_root_mean_squared_error: 1.2834 - val_loss: 2.4602 - val_y1_output_loss: 0.4029 - val_y2_output_loss: 2.1156 - val_y1_output_root_mean_squared_error: 0.6460 - val_y2_output_root_mean_squared_error: 1.4293
Epoch 107/500
614/614 [==============================] - 0s 139us/sample - loss: 2.2314 - y1_output_loss: 0.5467 - y2_output_loss: 1.7411 - y1_output_root_mean_squared_error: 0.7206 - y2_output_root_mea

Epoch 146/500
614/614 [==============================] - 0s 142us/sample - loss: 3.5447 - y1_output_loss: 0.8703 - y2_output_loss: 2.6527 - y1_output_root_mean_squared_error: 0.9358 - y2_output_root_mean_squared_error: 1.6337 - val_loss: 1.4469 - val_y1_output_loss: 0.2902 - val_y2_output_loss: 1.2024 - val_y1_output_root_mean_squared_error: 0.5413 - val_y2_output_root_mean_squared_error: 1.0742
Epoch 147/500
614/614 [==============================] - 0s 141us/sample - loss: 1.5527 - y1_output_loss: 0.3314 - y2_output_loss: 1.2452 - y1_output_root_mean_squared_error: 0.5694 - y2_output_root_mean_squared_error: 1.1084 - val_loss: 2.8158 - val_y1_output_loss: 0.5161 - val_y2_output_loss: 2.3474 - val_y1_output_root_mean_squared_error: 0.7241 - val_y2_output_root_mean_squared_error: 1.5138
Epoch 148/500
614/614 [==============================] - 0s 147us/sample - loss: 1.3507 - y1_output_loss: 0.3373 - y2_output_loss: 1.0070 - y1_output_root_mean_squared_error: 0.5823 - y2_output_root_mea

Epoch 187/500
614/614 [==============================] - 0s 141us/sample - loss: 1.3487 - y1_output_loss: 0.2513 - y2_output_loss: 1.0848 - y1_output_root_mean_squared_error: 0.5035 - y2_output_root_mean_squared_error: 1.0465 - val_loss: 1.3254 - val_y1_output_loss: 0.2744 - val_y2_output_loss: 1.0748 - val_y1_output_root_mean_squared_error: 0.5293 - val_y2_output_root_mean_squared_error: 1.0224
Epoch 188/500
614/614 [==============================] - 0s 142us/sample - loss: 1.8038 - y1_output_loss: 0.3942 - y2_output_loss: 1.3941 - y1_output_root_mean_squared_error: 0.6300 - y2_output_root_mean_squared_error: 1.1861 - val_loss: 1.2178 - val_y1_output_loss: 0.2229 - val_y2_output_loss: 1.0181 - val_y1_output_root_mean_squared_error: 0.4788 - val_y2_output_root_mean_squared_error: 0.9942
Epoch 189/500
614/614 [==============================] - 0s 137us/sample - loss: 1.2069 - y1_output_loss: 0.2773 - y2_output_loss: 0.9219 - y1_output_root_mean_squared_error: 0.5284 - y2_output_root_mea

Epoch 228/500
614/614 [==============================] - 0s 141us/sample - loss: 1.2229 - y1_output_loss: 0.2972 - y2_output_loss: 0.9149 - y1_output_root_mean_squared_error: 0.5470 - y2_output_root_mean_squared_error: 0.9611 - val_loss: 1.3999 - val_y1_output_loss: 0.2940 - val_y2_output_loss: 1.1124 - val_y1_output_root_mean_squared_error: 0.5411 - val_y2_output_root_mean_squared_error: 1.0522
Epoch 229/500
614/614 [==============================] - 0s 141us/sample - loss: 0.7518 - y1_output_loss: 0.1694 - y2_output_loss: 0.5799 - y1_output_root_mean_squared_error: 0.4119 - y2_output_root_mean_squared_error: 0.7630 - val_loss: 1.7093 - val_y1_output_loss: 0.2539 - val_y2_output_loss: 1.4626 - val_y1_output_root_mean_squared_error: 0.5037 - val_y2_output_root_mean_squared_error: 1.2065
Epoch 230/500
614/614 [==============================] - 0s 143us/sample - loss: 1.0754 - y1_output_loss: 0.2274 - y2_output_loss: 0.8498 - y1_output_root_mean_squared_error: 0.4779 - y2_output_root_mea

## Evaluate the Model and Plot Metrics

In [ ]:
# Test the model and print loss and mse for both outputs
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))

In [ ]:
# Plot the loss and mse
Y_pred = model.predict(norm_test_X)
plot_diff(test_Y[0], Y_pred[0], title='Y1')
plot_diff(test_Y[1], Y_pred[1], title='Y2')
plot_metrics(metric_name='y1_output_root_mean_squared_error', title='Y1 RMSE', ylim=6)
plot_metrics(metric_name='y2_output_root_mean_squared_error', title='Y2 RMSE', ylim=7)